In [31]:
import tensorflow as tf
from tensorflow.keras import layers, models, callbacks
import matplotlib.pyplot as plt
import numpy as np
import cv2 as cv

In [39]:
df = np.load("datasets/df.npz")

X = df["x"]
Y = df["y"]
Z = df["z"]

df = None
print(X.shape)

(1752, 224, 224)


In [40]:
def make_model():
    model = models.Sequential()
    shape=(X.shape[1], X.shape[2], 1)
    for i in range(0, 6):
        if i == 0:
            model.add(layers.Conv2D(32 + 2 ** (4 + i), (3, 3), strides=(2,2), activation='selu', input_shape=shape))
        else:
            model.add(layers.Conv2D(32 + 2 ** (4 + i), (3, 3), strides=(2,2), activation='relu'))
            

    model.add(layers.Flatten())
    model.add(layers.Dense(2, activation="relu"))

    model.summary()

    model.compile(optimizer='adam',
                  loss="mse",
                  metrics="mse")
    return model

model = make_model()

Model: "sequential_12"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_60 (Conv2D)          (None, 111, 111, 48)      480       
                                                                 
 conv2d_61 (Conv2D)          (None, 55, 55, 64)        27712     
                                                                 
 conv2d_62 (Conv2D)          (None, 27, 27, 96)        55392     
                                                                 
 conv2d_63 (Conv2D)          (None, 13, 13, 160)       138400    
                                                                 
 conv2d_64 (Conv2D)          (None, 6, 6, 288)         415008    
                                                                 
 conv2d_65 (Conv2D)          (None, 2, 2, 544)         1410592   
                                                                 
 flatten_11 (Flatten)        (None, 2176)            

In [41]:
checkpoint = callbacks.ModelCheckpoint(
    "dataset/tmp.h5",
    monitor='val_loss',
    save_best_only=True
)
history = model.fit(X, Y, epochs = 20, batch_size = 16, validation_split = 0.7, callbacks=[checkpoint])
model.load_weights("dataset/tmp.h5")

Epoch 1/20
33/33 [==============================] - 8s 229ms/step - loss: 0.2722 - mse: 0.2722 - val_loss: 0.1574 - val_mse: 0.1574
Epoch 2/20
33/33 [==============================] - 7s 226ms/step - loss: 0.1525 - mse: 0.1525 - val_loss: 0.0476 - val_mse: 0.0476
Epoch 3/20
33/33 [==============================] - 7s 230ms/step - loss: 0.0175 - mse: 0.0175 - val_loss: 0.0346 - val_mse: 0.0346
Epoch 4/20
33/33 [==============================] - 8s 239ms/step - loss: 0.0054 - mse: 0.0054 - val_loss: 0.0237 - val_mse: 0.0237
Epoch 5/20
33/33 [==============================] - 8s 254ms/step - loss: 0.0050 - mse: 0.0050 - val_loss: 0.0265 - val_mse: 0.0265
Epoch 6/20
33/33 [==============================] - 8s 243ms/step - loss: 0.0043 - mse: 0.0043 - val_loss: 0.0204 - val_mse: 0.0204
Epoch 7/20
33/33 [==============================] - 8s 242ms/step - loss: 0.0027 - mse: 0.0027 - val_loss: 0.0208 - val_mse: 0.0208
Epoch 8/20
33/33 [==============================] - 8s 242ms/step - loss: 0.

In [42]:
model.save_weights("weights/eye.h5")

In [43]:
preds = model.predict(X, verbose=0)

In [44]:
for i, pred in enumerate(preds):
    print(pred, Y[i])


[0.5306725 0.5607334] [0.51367188 0.47106481]
[0.5532322 0.5276076] [0.51367188 0.47106481]
[0.54946035 0.54013133] [0.51367188 0.47106481]
[0.5617744 0.5387375] [0.51367188 0.47106481]
[0.5617744 0.5387375] [0.51367188 0.47106481]
[0.5385207  0.53295714] [0.51367188 0.47106481]
[0.53118926 0.53951365] [0.51367188 0.47106481]
[0.53118926 0.53951365] [0.51367188 0.47106481]
[0.5316441 0.5156059] [0.51367188 0.47106481]
[0.5195699  0.47973582] [0.51432292 0.46643519]
[0.5195699  0.47973582] [0.51497396 0.46296296]
[0.5195699  0.47973582] [0.51497396 0.45833333]
[0.5356775 0.2914586] [0.51627604 0.45138889]
[0.5356775 0.2914586] [0.51627604 0.44328704]
[0.5581447 0.3221137] [0.51757812 0.42824074]
[0.5762365  0.35846046] [0.52148438 0.40046296]
[0.5762365  0.35846046] [0.5234375  0.37731481]
[0.5762365  0.35846046] [0.52408854 0.36226852]
[0.5565252 0.3788043] [0.52539062 0.34722222]
[0.5565252 0.3788043] [0.52669271 0.32986111]
[0.5595968  0.30198038] [0.52734375 0.31712963]
[0.55417275 